https://medium.com/agents-and-robots/the-bitter-truth-python-3-11-vs-cython-vs-c-performance-for-simulations-babc85cdfef5

In [1]:
from juliacall import Main as jl
%load_ext juliacall.ipython
# JuliaCall comes with its own Julia dependency file juliapkg.json
# however for binder it is much simpler to just reuse binder's installation mechanism
%julia Pkg.activate(Base.current_project())
%julia using PythonCall
%julia set_var(k, v) = @eval $(Symbol(k)) = $v

  Activating project at `~/Projects/Jolin.io/workshop-accelerate-Python-with-Julia`


set_var (generic function with 1 method)

In [2]:
import math
import random

from datetime import datetime
random.seed(datetime.now().timestamp())

WORLD_WIDTH = 2560
WORLD_HEIGHT = 1440
TIMESTEPS = 2000

jl.set_var("WORLD_WIDTH", WORLD_WIDTH)
jl.set_var("WORLD_HEIGHT", WORLD_HEIGHT)
jl.set_var("TIMESTEPS", TIMESTEPS)

2000

In [3]:
class PythonAgent():
    def __init__(self, x=None, y=None, world_width=0, world_height=0):
        super().__init__()

        # default values
        self.vmax = 2.0

        # initial position
        self.world_width = world_width
        self.world_height = world_height
        self.x = x if x else random.randint(0, self.world_width)
        self.y = y if y else random.randint(0, self.world_height)

        # initial velocity
        self.dx = 0
        self.dy = 0

        # inital values
        self.is_alive = True
        self.target = None
        self.age = 0
        self.energy = 0

    def update(self, food=()):
        self.age = self.age + 1

        # we can't move
        if self.vmax == 0:
            return

        # target is dead, don't chase it further
        if self.target and not self.target.is_alive:
            self.target = None

        # eat the target if close enough
        if self.target:
            squared_dist = (self.x - self.target.x) ** 2 + (self.y - self.target.y) ** 2
            if squared_dist < 400:
                self.target.is_alive = False
                self.energy = self.energy + 1

        # agent doesn't have a target, find a new one
        if not self.target:
            min_dist = 9999999
            min_agent = None
            for a in food:
                if a is not self and a.is_alive:
                    sq_dist = (self.x - a.x) ** 2 + (self.y - a.y) ** 2
                    if sq_dist < min_dist:
                        min_dist = sq_dist
                        min_agent = a
            if min_dist < 100000:
                self.target = min_agent

        # initalize 'forces' to zero
        fx = 0
        fy = 0

        # move in the direction of the target, if any
        if self.target:
            fx += 0.1*(self.target.x - self.x)
            fy += 0.1*(self.target.y - self.y)

        # update our direction based on the 'force'
        self.dx = self.dx + 0.05 * fx
        self.dy = self.dy + 0.05 * fy

        # slow down agent if it moves faster than it max velocity
        velocity = math.sqrt(self.dx ** 2 + self.dy ** 2)
        if velocity > self.vmax:
            self.dx = (self.dx / velocity) * (self.vmax)
            self.dy = (self.dy / velocity) * (self.vmax)

        # update position based on delta x/y
        self.x = self.x + self.dx
        self.y = self.y + self.dy

        # ensure it stays within the world boundaries
        self.x = max(self.x, 0)
        self.x = min(self.x, self.world_width)
        self.y = max(self.y, 0)
        self.y = min(self.y, self.world_height)

In [4]:
from agent import Agent as CythonAgent

In [11]:
def main(Agent):

    class Predator(Agent):
        def __init__(self, **kwargs):
            super().__init__(**kwargs)
            self.vmax = 2.5

    class Prey(Agent):
        def __init__(self, **kwargs):
            super().__init__(**kwargs)
            self.vmax = 2.0

    class Plant(Agent):
        def __init__(self, **kwargs):
            super().__init__(**kwargs)
            self.vmax = 0.0

            
    # open the ouput file
    # f = open('output.csv', 'w')
    # print(0, ',', 'Title', ',', 'Predator Prey Relationship / Example 02 / Cython', file=f)

    # create initial agents
    kwargs = dict(
        world_width=WORLD_WIDTH,
        world_height=WORLD_HEIGHT,
    )
    preys = [Prey(**kwargs) for i in range(10)]
    predators = [Predator(**kwargs) for i in range(10)]
    plants = [Plant(**kwargs) for i in range(100)]

    timestep = 0
    while timestep < TIMESTEPS:
        # update all agents
        # no need to update the plants; they do not move
        for a in preys:
            a.update(plants)
        
        for a in predators:
            a.update(preys)

        # handle eaten and create new plant
        plants = [p for p in plants if p.is_alive is True]
        plants = plants + [Plant(**kwargs) for i in range(2)]

        # handle eaten and create new preys
        preys = [p for p in preys if p.is_alive is True]

        for p in preys[:]:
            if p.energy > 5:
                p.energy = 0
                preys.append(Prey(x = p.x + random.randint(-20, 20), y = p.y + random.randint(-20, 20), **kwargs))

        # handle old and create new predators
        predators = [p for p in predators if p.age < 2000]

        for p in predators[:]:
            if p.energy > 10:
                p.energy = 0
                predators.append(Predator(x = p.x + random.randint(-20, 20), y = p.y + random.randint(-20, 20), **kwargs))

        # write data to output file
        #[print(timestep, ',', 'Position', ',', 'Predator', ',', a.x, ',', a.y, file=f) for a in predators]
        #[print(timestep, ',', 'Position',  ',', 'Prey', ',', a.x, ',', a.y, file=f) for a in preys]
        #[print(timestep, ',', 'Position',  ',', 'Plant', ',', a.x, ',', a.y, file=f) for a in plants]

        timestep = timestep + 1

    print(len(predators), len(preys), len(plants))

In [12]:
%timeit -r 3 main(PythonAgent)

2 271 1554
2 285 1355
10 206 1146
1 40 3486
5.27 s ± 2.07 s per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [13]:
%timeit main(CythonAgent)

6 297 913
0 56 3370
0 0 4055
4 182 1734
2 332 896
5 298 654
0 4 3983
4 218 1529
345 ms ± 28.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Full language switch

## Cython

In [14]:
import agent
%timeit agent.main(WORLD_WIDTH=WORLD_WIDTH, WORLD_HEIGHT=WORLD_HEIGHT, TIMESTEPS=TIMESTEPS)

(1, 245, 1872)
(4, 246, 1442)
(4, 295, 995)
(0, 180, 2434)
(0, 67, 3433)
(3, 346, 669)
(6, 304, 714)
(1, 118, 2726)
257 ms ± 50.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## C++

In [8]:
!g++ example.cpp -o example -std=c++11 -O3

zsh:1: command not found: g++


In [15]:
!perf stat -r 50 -ddd ./example $TIMESTEPS

ok 12, 249, 326
ok 12, 249, 326
ok 12, 249, 326
ok 12, 249, 326
ok 12, 249, 326
ok 12, 249, 326
ok 12, 249, 326
ok 12, 249, 326
ok 12, 249, 326
ok 12, 249, 326
ok 12, 249, 326
ok 12, 249, 326
ok 8, 335, 389
ok 8, 335, 389
ok 8, 335, 389
ok 8, 335, 389
ok 8, 335, 389
ok 8, 335, 389
ok 8, 335, 389
ok 8, 335, 389
ok 8, 335, 389
ok 8, 335, 389
ok 8, 335, 389
ok 8, 335, 389
ok 1, 253, 1692
ok 1, 253, 1692
ok 1, 253, 1692
ok 1, 253, 1692
ok 1, 253, 1692
ok 1, 253, 1692
ok 1, 253, 1692
ok 1, 253, 1692
ok 1, 253, 1692
ok 1, 253, 1692
ok 1, 253, 1692
ok 2, 315, 1096
ok 2, 315, 1096
ok 2, 315, 1096
ok 2, 315, 1096
ok 2, 315, 1096
ok 2, 315, 1096
ok 2, 315, 1096
ok 2, 315, 1096
ok 2, 315, 1096
ok 2, 315, 1096
ok 2, 315, 1096
ok 2, 315, 1096
ok 2, 315, 1096
ok 0, 0, 4060
ok 0, 0, 4060

 Performance counter stats for './example 2000' (50 runs):

              5,28 msec task-clock:u                     #    0,070 CPUs utilized            ( +- 53,25% )
                 0      context-switches:u      

## Julia

In [16]:
%%julia
Base.@kwdef mutable struct Agent
    vmax::Float64 = 2.5
    world_width::Int = 10
    world_height::Int = 10
    x::Int = rand(0:world_width)
    y::Int = rand(0:world_height)
    
    # initial velocity
    dx::Float64 = 0.0
    dy::Float64 = 0.0

    # inital values
    is_alive::Bool = true
    target::Union{Nothing, Agent} = nothing
    age::Int = 0
    energy::Int = 0
end

Predator(; kwargs...) = Agent(; vmax = 2.5, kwargs...)
Prey(; kwargs...) = Agent(; vmax = 2.0, kwargs...)
Plant(; kwargs...) = Agent(; vmax = 0.0, kwargs...)

Plant (generic function with 1 method)

In [17]:
%%julia
function update!(self::Agent, food::Vector{Agent})
    self.age = self.age + 1

    # we can't move
    if self.vmax == 0.0
        return
    end

    # target is dead, don't chase it further
    if self.target !== nothing && !self.target.is_alive
        self.target = nothing
    end
    
    # eat the target if close enough
    if self.target !== nothing
        squared_dist = (self.x - self.target.x) ^ 2 + (self.y - self.target.y) ^ 2
        if squared_dist < 400
            self.target.is_alive = false
            self.energy = self.energy + 1
        end
    # agent doesn't have a target, find a new one
    else
        min_dist = 9999999
        min_agent = nothing
        for a in food
            if a !== self && a.is_alive
                sq_dist = (self.x - a.x) ^ 2 + (self.y - a.y) ^ 2
                if sq_dist < min_dist
                    min_dist = sq_dist
                    min_agent = a
                end
            end
        end
        if min_dist < 100000
            self.target = min_agent
        end
    end

    # initalize 'forces' to zero
    fx = 0.0
    fy = 0.0

    # move in the direction of the target, if any
    if self.target !== nothing
        fx += 0.1 * (self.target.x - self.x)
        fy += 0.1 * (self.target.y - self.y)
    end

    # update our direction based on the 'force'
    self.dx = self.dx + 0.05 * fx
    self.dy = self.dy + 0.05 * fy

    # slow down agent if it moves faster than it max velocity
    velocity = sqrt(self.dx ^ 2 + self.dy ^ 2)
    if velocity > self.vmax
        self.dx = (self.dx / velocity) * (self.vmax)
        self.dy = (self.dy / velocity) * (self.vmax)
    end

    # update position based on delta x/y
    self.x = self.x + Int(round(self.dx))
    self.y = self.y + Int(round(self.dy))

    # ensure it stays within the world boundaries
    self.x = max(self.x, 0)
    self.x = min(self.x, self.world_width)
    self.y = max(self.y, 0)
    self.y = min(self.y, self.world_height)
end

update! (generic function with 1 method)

In [18]:
%%julia
function main()
    kwargs = (
        world_width = WORLD_WIDTH,
        world_height = WORLD_HEIGHT,
    )
    preys = [Prey(; kwargs...) for i in 1:10]
    predators = [Predator(; kwargs...) for i in 1:10]
    plants = [Plant(; kwargs...) for i in 1:100]

    timestep = 0
    while timestep < TIMESTEPS
        # update all agents
        #[f.update([]) for f in plants]  # no need to update the plants; they do not move
        for a in preys
            update!(a, plants)
        end
        for a in predators
            update!(a, preys)
        end

        # handle eaten and create new plant
        # plants = [p for p in plants if p.is_alive]
        filter!(p -> p.is_alive, plants)
        append!(plants, [Plant(; kwargs...) for i in 1:2])

        # handle eaten and create new preys
        # preys = [p for p in preys if p.is_alive]
        filter!(p -> p.is_alive, preys)
        for p in preys
            if p.energy > 5
                p.energy = 0
                push!(preys, Prey(; x = p.x + rand(-20:20), y = p.y + rand(-20:20), kwargs...))
            end
        end

        # handle old and create new predators
        # predators = [p for p in predators if p.age < 2000]
        filter!(p -> p.age < 2000, predators)
        for p in predators
            if p.energy > 10
                p.energy = 0
                push!(predators, Predator(; x = p.x + rand(-20:20), y = p.y + rand(-20:20), kwargs...))
            end
        end

        # write data to output file
        #[print(timestep, ',', 'Position', ',', 'Predator', ',', a.x, ',', a.y, file=f) for a in predators]
        #[print(timestep, ',', 'Position',  ',', 'Prey', ',', a.x, ',', a.y, file=f) for a in preys]
        #[print(timestep, ',', 'Position',  ',', 'Plant', ',', a.x, ',', a.y, file=f) for a in plants]

        timestep += 1
    end
    println("$(length(predators)), $(length(preys)), $(length(plants))")
end

main (generic function with 1 method)

In [19]:
%julia using BenchmarkTools
%julia @btime main()

1, 55, 3477
7, 93, 2476
7, 296, 675
5, 284, 1054
3, 289, 1291
4, 388, 362
3, 195, 1940
2, 360, 546
4, 408, 229
0, 16, 3826
2, 113, 2767
0, 208, 2198
2, 363, 502
6, 268, 1194
4, 375, 373
6, 352, 321
1, 103, 3068
3, 402, 264
5, 302, 1101
2, 308, 1292
5, 352, 324
1, 135, 2700
3, 396, 349
2, 331, 854
3, 390, 255
2, 230, 1818
3, 112, 2721
0, 1, 4007
0, 3, 3962
4, 296, 1045
5, 299, 1032
0, 57, 3577
1, 397, 524
2, 230, 1725
0, 204, 2238
2, 388, 503
2, 306, 1013
2, 242, 1706
3, 241, 1693
0, 2, 4004
3, 397, 235
4, 300, 1041
3, 278, 1160
4, 274, 1271
4, 378, 377
5, 276, 1228
1, 379, 488
4, 313, 874
0, 362, 976
4, 370, 376
6, 172, 1567
6, 294, 661
3, 379, 521
5, 329, 580
3, 387, 489
4, 377, 247
0, 420, 445
0, 259, 1875
3, 266, 1398
4, 273, 1371
1, 300, 1315
2, 356, 861
2, 379, 414
4, 232, 1554
2, 273, 1255
2, 380, 283
1, 289, 1463
7, 337, 327
0, 0, 3992
8, 346, 242
5, 392, 234
1, 151, 2646
1, 118, 2849
2, 288, 1326
0, 61, 3280
2, 401, 434
8, 233, 996
2, 370, 525
3, 283, 1190
4, 223, 1654
9, 336, 

In [20]:
%julia @benchmark main()

1, 179, 2482
2, 267, 1509
0, 425, 279
0, 250, 1958
5, 366, 387
0, 256, 1778
0, 293, 1596
3, 373, 322
6, 330, 299
4, 252, 1194
5, 358, 306
7, 341, 275
1, 341, 985
2, 266, 1562
2, 315, 1140
3, 397, 241
1, 32, 3491
4, 382, 420
5, 319, 536
0, 0, 4031
5, 300, 805
2, 251, 1567
0, 2, 4011
3, 198, 2032
5, 336, 742
1, 267, 1619
6, 157, 2100
1, 318, 1262
0, 295, 1515
1, 363, 1004
1, 239, 1755
1, 160, 2536
9, 318, 281
0, 3, 3977
3, 238, 1701
0, 91, 2970
3, 335, 702
1, 150, 2454
3, 335, 853
1, 0, 3963
4, 350, 522
4, 378, 363
5, 303, 814
0, 226, 2131
4, 375, 302
3, 363, 500
0, 2, 4055
1, 186, 2346
1, 249, 1703
1, 377, 836
2, 244, 1728
3, 293, 1181
5, 368, 318
6, 198, 1522
1, 1, 3864
0, 199, 2382
3, 362, 463
4, 285, 1100
2, 291, 1245
3, 338, 786
2, 271, 1477
2, 275, 1294
6, 324, 262
6, 336, 258
1, 418, 358
1, 144, 2704
0, 71, 3324
1, 200, 2274
0, 2, 4033
2, 420, 374
3, 241, 1576
5, 391, 205
4, 297, 811
1, 304, 1366
1, 254, 1690
3, 342, 684
0, 293, 1479
6, 290, 616
3, 389, 269
1, 393, 506
1, 323, 128

BenchmarkTools.Trial: 72 samples with 1 evaluation.
 Range (min … max):   8.974 ms … 138.448 ms  ┊ GC (min … max): 0.00% … 2.91%
 Time  (median):     75.216 ms               ┊ GC (median):    1.09%
 Time  (mean ± σ):   70.001 ms ±  24.239 ms  ┊ GC (mean ± σ):  1.50% ± 0.97%

                                ▅  █  ▅                         
  ▅▄▄▁▁▁▁▅▁▁▄▁▁▅▁▁▁▅▁▁▄▄▅▅▅▄▄▄▅▇█▅▇█▇▇█▇▅▄▁▁▁▅▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▄ ▁
  8.97 ms         Histogram: frequency by time          133 ms <

 Memory estimate: 1.52 MiB, allocs estimate: 11827.